In [ ]:
# Add directory above current directory to path
import sys

sys.path.insert(0, "..")

from sunflower.sunflower.song_loader import Song, load_from_disk
from  sunflower.sunflower.song_analyzer import SongAnalyzer
from  sunflower.sunflower.utils import export_wav
from  sunflower.sunflower.song_visualizer import visualize_waveform
import plotly.graph_objects as go
import numpy as np
import librosa
import soundfile as sf

# Autoreloading for easier development

%load_ext autoreload
%autoreload 2

## Song loading & analysis

In [ ]:
# Loading example file

raw_audio, extension = load_from_disk("assets/laylow_casting_slowed.mp3")

song = Song(raw_audio, extension)

song.print_attributes()

In [ ]:
# Analyzing song

song_analyzer = SongAnalyzer(song)
song_analyzer.detect_tempo()

print(song_analyzer.tempo)

In [ ]:
song_analyzer.beat_frames

## Waveform visualization

In [ ]:
visualize_waveform(song)

In [ ]:
stereo = False
rate = 30

left = [point[0] for point in song.waveform[0::rate]]
right = [point[1] for point in song.waveform[0::rate]]
mono = (np.array(left) + np.array(right)) / 2
x = np.arange(0, len(mono)) * rate / song.sr

if stereo:
    data = [
        go.Scatter(x=x, y=left, name="Left"),
        go.Scatter(x=x, y=right, name="Right"),
    ]
else:
    data = [go.Scatter(x=x, y=mono, name="Mono")]

fig = go.Figure(data=data)

beats = librosa.frames_to_time(song_analyzer.beat_frames, song.sr)

fig.add_trace(
    go.Scatter(
        x=beats,
        y=np.zeros(len(beats)),
        mode="markers",
        marker_symbol="diamond-tall",
        marker_size=10,
        name="Beat Rythm"
    )
)

fig.update_layout(title=f"Waveform Visualizer <br>BPM :{round(song_analyzer.tempo,0)}")
fig.show()